# Pasta

## Index
1. [Instantiate model class](#Instantiate-model-class)
2. [Define clock metadata](#Define-clock-metadata)
3. [Download clock dependencies](#Download-clock-dependencies)
5. [Load features](#Load-features)
6. [Load weights into base model](#Load-weights-into-base-model)
7. [Load reference values](#Load-reference-values)
8. [Load preprocess and postprocess objects](#Load-preprocess-and-postprocess-objects)
10. [Check all clock parameters](#Check-all-clock-parameters)
10. [Basic test](#Basic-test)
11. [Save torch model](#Save-torch-model)
12. [Clear directory](#Clear-directory)


Let's first import some packages:

In [1]:
import os
import inspect
import shutil
import json
import torch
import pandas as pd
import pyaging as pya

## Instantiate model class

In [2]:
def print_entire_class(cls):
    source = inspect.getsource(cls)
    print(source)

print_entire_class(pya.models.Pasta)


class Pasta(pyagingModel):
    def __init__(self):
        super().__init__()

    @staticmethod
    def _rank_average(values):
        """
        Assign average ranks (1-based) per vector, handling ties.
        """
        sorted_vals, sorted_idx = torch.sort(values)
        ranks = torch.empty_like(sorted_vals, dtype=values.dtype)

        n = values.numel()
        start = 0
        while start < n:
            end = start + 1
            while end < n and sorted_vals[end] == sorted_vals[start]:
                end += 1
            avg_rank = (start + end - 1) / 2.0 + 1.0
            ranks[sorted_idx[start:end]] = avg_rank
            start = end

        return ranks

    def preprocess(self, x):
        """
        Fill missing values with the global median then rank-normalize per sample.
        """
        median = torch.nanmedian(x)
        if torch.isnan(median):
            median = torch.tensor(0.0, device=x.device, dtype=x.dtype)
        x = torch.where(torch.isnan(x), me

In [3]:
model = pya.models.Pasta()

## Define clock metadata

In [4]:
model.metadata["clock_name"] = 'pasta'
model.metadata["data_type"] = 'transcriptomics'
model.metadata["species"] = 'Homo sapiens'
model.metadata["year"] = 2025
model.metadata["approved_by_author"] = '✅'
model.metadata["citation"] = "Salignon, Jerome, et al. \"Pasta, an age-shift transcriptomic clock, maps the chemical and genetic determinants of aging and rejuvenation.\" bioRxiv (2025): 2025-06."
model.metadata["doi"] = "https://doi.org/10.1101/2025.06.04.657785"
model.metadata["research_only"] = None
model.metadata["notes"] = "Rank-normalized transcriptomic clock."


## Download clock dependencies

#### Download coefficient file

In [5]:
coeff_url = "https://raw.githubusercontent.com/bio-learn/biolearn/master/biolearn/data/Pasta.csv"
os.system(f"curl -L {coeff_url} -o Pasta.csv")


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  322k  100  322k    0     0  1174k      0 --:--:-- --:--:-- --:--:-- 1176k


0

## Load features

#### From CSV file

In [6]:
coeffs = pd.read_csv('Pasta.csv')
coeffs['feature'] = coeffs['GeneID']
coeffs['coefficient'] = coeffs['CoefficientTraining']

model.features = coeffs['feature'].tolist()


## Load weights into base model

#### From CSV file

In [7]:
weights = torch.tensor(coeffs['coefficient'].tolist()).unsqueeze(0)
intercept = torch.tensor([0.0])


#### Linear model

In [8]:
base_model = pya.models.LinearModel(input_dim=len(model.features))

base_model.linear.weight.data = weights.float()
base_model.linear.bias.data = intercept.float()

model.base_model = base_model

## Load reference values

In [9]:
model.reference_values = [float("nan")] * len(model.features)

## Load preprocess and postprocess objects

In [10]:
model.preprocess_name = "median_fill_and_rank_normalization"
model.preprocess_dependencies = None


In [11]:
model.postprocess_name = "scale_and_shift"
model.postprocess_dependencies = [-4.76348378687217, -0.0502893445253186]


## Check all clock parameters

In [12]:
pya.utils.print_model_details(model)


%==================================== Model Details ====================================%
Model Attributes:

training: True
metadata: {'approved_by_author': '✅',
 'citation': 'Salignon, Jerome, et al. "Pasta, an age-shift transcriptomic '
             'clock, maps the chemical and genetic determinants of aging and '
             'rejuvenation." bioRxiv (2025): 2025-06.',
 'clock_name': 'pasta',
 'data_type': 'transcriptomics',
 'doi': 'https://doi.org/10.1101/2025.06.04.657785',
 'notes': 'Rank-normalized transcriptomic clock.',
 'research_only': None,
 'species': 'Homo sapiens',
 'version': None,
 'year': 2025}
reference_values: [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]... [Total elements: 8113]
preprocess_name: 'median_fill_and_rank_normalization'
preprocess_dependencies: None
postprocess_name: 'scale_and_shift'
postprocess_dependencies: [-4.76348378687217, -0.050289344525318

## Basic test

In [13]:
torch.manual_seed(42)
input = torch.randn(10, len(model.features), dtype=float)
model.eval()
model.to(float)
pred = model(input)
pred

tensor([[-12.0950],
        [-36.7366],
        [  2.5806],
        [ 29.9285],
        [-21.5258],
        [ -3.6489],
        [-39.3746],
        [ 36.6380],
        [ 27.4081],
        [-33.9218]], dtype=torch.float64, grad_fn=<AddBackward0>)

## Save torch model

In [14]:
torch.save(model, f"../weights/{model.metadata['clock_name']}.pt")

## Clear directory
<a id="10"></a>

In [15]:
# Function to remove a folder and all its contents
def remove_folder(path):
    try:
        shutil.rmtree(path)
        print(f"Deleted folder: {path}")
    except Exception as e:
        print(f"Error deleting folder {path}: {e}")

# Get a list of all files and folders in the current directory
all_items = os.listdir('.')

# Loop through the items
for item in all_items:
    # Check if it's a file and does not end with .ipynb
    if os.path.isfile(item) and not item.endswith('.ipynb'):
        os.remove(item)
        print(f"Deleted file: {item}")
    # Check if it's a folder
    elif os.path.isdir(item):
        remove_folder(item)

Deleted file: EPIC_salas_18_reference.csv
Deleted file: Pasta.csv
